In [ ]:
import numpy as np
import yaml
import pandas as pd
import json
from tqdm import tqdm

In [2]:
SLA = 250
key_name = "rt_ti_u_bound_c90_p95" #rt_ti_u_bound_c90_p95 responseTimes95th
BASE_DIR = '/home/vahid/Dropbox/data/swarm-manager-data/results/nodejs_bookstore/'

In [3]:
workloads = []
for ap in [0.1,0.3,0.5,0.7,0.9]:
    for vus in [400,475,550,625]:
        workloads.append(str(vus)+"_120_"+str(ap)+"_10")      
#"300_120_0.3_10",
#"400_120_0.5_10"
# workloads = ["400_120_0.5_10", "500_120_0.65_10", "600_120_0.3_10","700_120_0.5_10"]

In [4]:
workloads

['400_120_0.1_10',
 '475_120_0.1_10',
 '550_120_0.1_10',
 '625_120_0.1_10',
 '400_120_0.3_10',
 '475_120_0.3_10',
 '550_120_0.3_10',
 '625_120_0.3_10',
 '400_120_0.5_10',
 '475_120_0.5_10',
 '550_120_0.5_10',
 '625_120_0.5_10',
 '400_120_0.7_10',
 '475_120_0.7_10',
 '550_120_0.7_10',
 '625_120_0.7_10',
 '400_120_0.9_10',
 '475_120_0.9_10',
 '550_120_0.9_10',
 '625_120_0.9_10']

In [5]:
# cpu_util_rules = sorted(['cpu90>50','cpu90>60','cpu90>70','cpu90>80','cpu90>90',
#                          'cpu95>50','cpu95>60','cpu95>70','cpu95>80','cpu95>90',
#                          'cpu.mean>50','cpu.mean>60','cpu.mean>70','cpu.mean>80','cpu.mean>90'])
strategies = {
    'adfc-ti':'/AddDifferentFractionalCPUcores/adfc_utilization_ti_95_'+str(SLA)+'.yml',
    'mc-adfc-ti':'/AddDifferentFractionalCPUcores/mc_adfc_utilization_ti_95_'+str(SLA)+'.yml',

    'ppeus-ti':'/PerPathEUBasedScaling/ppeus_ti_equal_steps_950_'+str(SLA)+'.yml',
    'mc-ppeus-ti':'/PerPathEUBasedScaling/mc_ppeus_ti_equal_steps_950_'+str(SLA)+'.yml',
    
    'afc-0.33-ti':'/AddFractionalCPUcores/afc_0.33_ti_95_'+str(SLA)+'.yml'
}


In [6]:
def does_specs_meet_sla(config):
    return config['auth']['responseTimes']['total'][key_name] < SLA and \
            config['books']['responseTimes']['total'][key_name] < SLA

temp_dict_cpu = {'workloads':[]}
temp_dict_steps = {'workloads':[]}
for strategy, file_path in strategies.items():
    temp_dict_cpu[strategy] = []
    temp_dict_steps[strategy] = []
    for workload in workloads:
        if len(workloads) != len(temp_dict_cpu['workloads']): temp_dict_cpu['workloads'].append(workload)
        if len(workloads) != len(temp_dict_steps['workloads']): temp_dict_steps['workloads'].append(workload)
        
        meetsSLA = False
        cpu_count = 0
        requiredSteps = 0
        with open(BASE_DIR + workload + file_path) as f:
            data = yaml.load(f, Loader=yaml.FullLoader)
        for _step in data['steps']:
            requiredSteps += 1
            cpu_count = _step['specs']['auth']['replicaCount'] * _step['specs']['auth']['CPULimits'] + _step['specs']['books']['replicaCount'] * _step['specs']['books']['CPULimits'] + _step['specs']['gateway']['replicaCount'] * _step['specs']['gateway']['CPULimits']
            if does_specs_meet_sla(_step['info']):
                meetsSLA = True
                break
        
        if not meetsSLA: 
            requiredSteps = 0
            cpu_count = 0
            temp_dict_cpu[strategy].append('-')
            temp_dict_steps[strategy].append('-')
        else:
#             temp_dict[strategy].append((np.round(cpu_count,2), requiredSteps))
            temp_dict_cpu[strategy].append(np.round(cpu_count,2))
            temp_dict_steps[strategy].append(requiredSteps)

for col, values in temp_dict_cpu.items():
    print(col, len(values))
        

workloads 20
adfc-ti 20
mc-adfc-ti 20
ppeus-ti 20
mc-ppeus-ti 20
afc-0.33-ti 20


In [7]:
df = pd.DataFrame(temp_dict_cpu)
print(df.head(25))
print('average number of CPU cores required to find the configuration which meet the SLA')
df.mean(axis=0)

         workloads  adfc-ti  mc-adfc-ti  ppeus-ti  mc-ppeus-ti  afc-0.33-ti
0   400_120_0.1_10     6.93        7.00      7.07         7.07         8.28
1   475_120_0.1_10     7.86        8.00      7.84         7.84         9.27
2   550_120_0.1_10     8.93        9.93     10.60        10.60        11.25
3   625_120_0.1_10    10.86       11.86     11.36        12.36        13.89
4   400_120_0.3_10     7.04        7.04      3.90         4.90         6.96
5   475_120_0.3_10     7.82        7.82      7.63         8.63         7.95
6   550_120_0.3_10     9.06        9.06      9.36         9.36         9.60
7   625_120_0.3_10    10.07       10.07     11.09        13.09        11.25
8   400_120_0.5_10     6.67        7.00      3.72         3.72         6.96
9   475_120_0.5_10     7.67        8.00      7.42         8.42         7.62
10  550_120_0.5_10     8.67        9.00      9.12         9.11         9.27
11  625_120_0.5_10    10.00       10.60      9.81        10.81        10.26
12  400_120_

adfc-ti        8.3065
mc-adfc-ti     8.5850
ppeus-ti       7.9170
mc-ppeus-ti    8.5665
afc-0.33-ti    9.7155
dtype: float64

In [8]:
df = pd.DataFrame(temp_dict_steps)
print(df.head(25))
print('average number of steps required to find the configuration which meet the SLA')
df.mean(axis=0)

         workloads  adfc-ti  mc-adfc-ti  ppeus-ti  mc-ppeus-ti  afc-0.33-ti
0   400_120_0.1_10        5           5         3            3            8
1   475_120_0.1_10        6           6         3            3            9
2   550_120_0.1_10        7           8         4            4           11
3   625_120_0.1_10        9          10         4            5           14
4   400_120_0.3_10        5           5         1            2            6
5   475_120_0.3_10        6           6         3            3            7
6   550_120_0.3_10        7           7         3            3            9
7   625_120_0.3_10        8           8         4            6           11
8   400_120_0.5_10        5           5         1            1            5
9   475_120_0.5_10        6           6         3            3            6
10  550_120_0.5_10        7           7         3            3            8
11  625_120_0.5_10        8           9         3            4            9
12  400_120_

adfc-ti        6.60
mc-adfc-ti     6.75
ppeus-ti       2.80
mc-ppeus-ti    3.30
afc-0.33-ti    9.65
dtype: float64

In [26]:
workloads = ["400_120_0.5_10"]
def does_specs_meet_sla(config):
    return config['auth']['responseTimes']['total'][key_name] < SLA and \
            config['books']['responseTimes']['total'][key_name] < SLA

for workload in workloads:
    cpu_counts = []
    step_counts = []
    meetsSLA = False
    cpu_count = 0
    requiredSteps = -1 # the first step is always 1,1,1
    with open(BASE_DIR + workload + '/MOBO/mobo_CPU_split_mc.yml') as f:
        data = yaml.load(f, Loader=yaml.FullLoader)
    for _step in data['steps']:
        meetsSLA = False
        requiredSteps += 1
        cpu_count = _step['specs']['auth']['replicaCount'] * _step['specs']['auth']['CPULimits'] + _step['specs']['books']['replicaCount'] * _step['specs']['books']['CPULimits'] + _step['specs']['gateway']['replicaCount'] * _step['specs']['gateway']['CPULimits']
        if does_specs_meet_sla(_step['info']):
            meetsSLA = True

        if not meetsSLA: 
            cpu_count = 0
        else:
            cpu_counts.append(np.round(cpu_count,2))
            step_counts.append(requiredSteps)
    print(workload)
    best_cpu_count = 10000
    best_pair = (0,0)
    for i in zip(cpu_counts,step_counts):
        print(i)
        if i[0] < best_cpu_count:
            best_pair = i
            best_cpu_count = i[0]
    print('best:', best_pair)
    print('==========')

workload: 400_120_0.5_10
(12.45, 2)
(16.13, 5)
(17.28, 6)
(16.4, 10)
(12.47, 20)
best: (12.45, 2)
